In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
    "num p_iwl >= 0.5",
    "p_iwl given context contains query class",
    "loss"
    # "similarity",
]

# Transformer
stats_keys = [
    "accuracy",
    "context contains query class",
    "loss"
]

In [6]:
repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
# results_dir = "/Users/chanb/research/ualberta/icl/scratch/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/evaluation_results"
results_dir = "/Users/chanb/research/ualberta/icl/cc_results/noisy_labels_0.01/evaluation_results"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

In [7]:
# variant_name = "synthetic-simple_icl"
# variant_name = "synthetic-transformer"
# variant_name = "synthetic-transformer-context_len"
# variant_name = "synthetic-transformer-large_num_low_freq"
variant_name = "synthetic-transformer-large_num_low_freq-input_noise_0.2"
# variant_name = "synthetic-transformer-no_noise"
# variant_name = "synthetic-transformer-noisy_inputs_0.2"
# variant_name = "synthetic-transformer-noisy_labels_0.01"
# variant_name = "synthetic-transformer-noisy_labels_0.1"
# variant_name = "synthetic-transformer-num_relevant_contexts"

os.makedirs(
    os.path.join(repo_path, "plot_utils/plots/agg_stats"),
    exist_ok=True
)

In [ ]:
stats_file = os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name))
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if variant_name == "synthetic-simple_icl":
        all_dirs = [
            "synthetic-alpha",
            "synthetic-ic_predictor",
            "synthetic-iw_predictor",
        ]
    else:
        all_dirs = [variant_name]

    for curr_dir in all_dirs:
        curr_results_dir = os.path.join(results_dir, curr_dir)
        if repo_path.startswith("/Users"):
            it = tqdm(os.listdir(curr_results_dir))
        else:
            it = os.listdir(curr_results_dir)

        for run_name in it:
            tic = timeit.default_timer()

            prefix = "-".join(run_name.split("-")[:-9])
            seed = run_name.split("-")[-9]

            data = pickle.load(open(os.path.join(curr_results_dir, run_name), "rb"))
            for eval_name in data["stats"]:
                for stats_key in stats_keys:
                    if not stats_key in data["stats"][eval_name]:
                        continue

                    factor = 1
                    if stats_key == "accuracy":
                        factor = 1 / 100

                    stats.append(
                        dict(
                            dirname=curr_dir,
                            variant=prefix,
                            seed=seed,
                            eval_name=eval_name,
                            stats_key=stats_key,
                            stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                            **{
                                "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                            },
                        )
                    )

            toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name)))

In [ ]:
len(stats)

In [ ]:
stats.info()

In [ ]:
stats